# ASTR302 Lab 2: Statistical Distributions and Uncertainties

In this Lab you will continue to explore statistical distributions and develop an understanding of how they are used to estimate measurement uncertainties.

## Statistical Distribution Review

Previously, we explored the uniform, Gaussian (or Normal), Binomial, and Poisson distributions. The latter three all have wide use in determining measurement uncertianties or outcomes.

You should have a good understanding of which situations are related to the use of the Binomial and Poisson distributions from Lab 1. 

The Gaussian distribution is most closely related to the distribution of expected measurement deviations. This makes intuitive sense in that we expect our measurements to be unbiased with respect to the underlying correct value (so the deviations are symetrically centered on zero deviation) and that large deviations are rarer than small ones. Exactly how rare or not we will calculate next.

## Gaussians and the Interpretaton of $\sigma$

Before we start, lets import the packages you will be needing for this Lab.


In [ ]:
import numpy as np
import scipy.integrate as integrate   # to help us do integrals

Recall that the Gaussian distribution is
$$P(x) = \frac{1}{\sigma\sqrt{2\pi}} e^{-(x-\mu)^2/2\sigma^2}$$
where $\mu$ is the mean value (the true value if **there are no systematic errors** and $\sigma$ is the **standard deviation** or dispersion. The coefficient is there to ensure that the integral from $-\infty$ to $\infty$ is 1 (as it should be because there is a 100% chance of the measurement being between $-\infty$ to $\infty$). Therefore, $\sigma$ is clearly the parameter of interest that describes how far 'off' our measurements can be.

Let's find out what fraction of the time we can expect our measurement to be 1$\sigma$ or less from the true value. To do this we can just integrate $P(x)$ between $\pm 1 \sigma$.

In [ ]:
sigma = 1
integral = integrate.quad(lambda x: 1/np.sqrt(2*np.pi)*np.exp(-x**2/2), -1, 1)
print(f'Fraction of area under Gaussian between +- 1 sigma = {integral[0]:.3f}')
print(f'Likelihood of having a chance measurement beyond +- 1 sigma = {1-integral[0]:.3f}')

A measurement that has a low likelihood of agreeing with the expectation is taken as evidence of a problem with the expectation. For example, it can be used out to rule out the specific model on which that expectation was based. A 32% chance, however, is not particularly compelling - it isn't so unlikely that the disagreement is just due to random chance. By convention in astronomy we often take a 3$\sigma$ deviation as being compelling.

<div class="alert alert-info">What is the likelihood of such a deviation being due to random chance? Answer below. </div> 

In [ ]:
Answer:

For particularly important results, the threshold may be set even higher.

An important property of this line of thinking is that the likelihood is not dependent on the specific value of $\sigma$ as long as you quote the deviation in terms of $\sigma$ (i.e. a '3$\sigma$ deviation'). 

<div class="alert alert-info"> Confirm that is the case by changing the value of $\sigma$ to two - and don't forget to change the equation in the integrate statement.</div>

It is always a good thing when there are multiple ways to calculate an answer to a question. 

Integrals can in general also be evaluated using random numbers. Such technniques are generally dubbed 'Monte Carlo' in reference to the element of chance (and the famous casino of Monte Carlo - see https://www.montecarlosbm.com/en/casino-monaco/casino-monte-carlo). 

In this case we can just draw from a Gaussian distribution (which you have already done in Lab 1) and evaluate what fraction of the time is the draw within 1$\sigma$. 

<div class="alert alert-info">Confirm your previous results for 1 and 3 $\sigma$. If they are not quite the same as before, why and what might you change? Answer below. </div>

In [ ]:
Answer:

In [ ]:
sample = np.random.normal(0,1,1000)   # we have changed sigma to 1

...

print(f'Fraction of area under Gaussian between +- 1 sigma = {len(sample_within_1_sigma)/len(sample):.3f}')
print(f'Likelihood of having a measurement beyond +- 1 sigma = {1-len(sample_within_1_sigma)/len(sample):.3f}')

## Determining $\sigma$

If we know the value of $\sigma$, then we can calculate the likelihood of a particular measurement - **however we rarely know $\sigma$ in advance.** To estimate $\sigma$ we turn to the standard deviation of our measurements. If we have enough meaurements and if the measurement excursions are drawn from a Gaussian distribution, then the standard deviation will be a fair approximation of $\sigma$. 

Recall that the standard deviation is calcluated using
$$S^2 = \frac{1}{N}\sum_{i=1}^{N} (x_i-\mu)^2,$$
but fortuntely numpy has a built-in function to do this.

In [ ]:
standard_dev = np.std(sample)
print(f'The standard deviation of the set of Gaussian draws = {standard_dev:.3f}')

Did you get the result you expected?

<div class="alert alert-info"> Now, lets assume you have a set of measurements (included in the cell below as the array measurements). 
What you would quote as your result? 
The convention is to quote the mean value $\pm 1\sigma$. So remember that most uncertainty ranges that you see only include about 2/3rds of the expected measurements. Fill in the cell below. </div>

In [ ]:
# calculate mean value and standard deviation
# np.mean is the numpy function for mean

measurements = np.array([5.99792596,11.06015541,11.65410636,19.80187928,6.759849,13.71022563,
16.6081179,13.88738697,13.98100988,12.44655947])

#
#
#

The Gaussian distribution is appropriate when the distribution is not limited by other factors. An example where the distribution is limited is a counting experiment because the counts cannot be $<$ 0. This situation can become particular extreme if the expected counts are quite small, i.e. near 0. For this we use the Poisson distribution and the uncertainties can be calculated in a similar manner (using either the integral or sampling approaches) but there is also a simple prescritpion that works roughly, which is that $\sigma = \sqrt{N}$, where $N$ is the number of events. For example, if the mean is 10 events, then you can quote 10$\pm \sqrt{10}$. Because photon flux rates tend to be small, Poissonian error estimates are often used in this context - but be careful, because they are not quite mathematically correct.

## Propagating Uncertainties

You now know how to take your measurements and quote a value, its uncertainty, and therefore the likelihood of getting such a value in comparison either to another measurement or a theoretical model. But what happens if what you are really after is a mathematical combination of two measurements. In others words, $(A \pm \sigma_A) + (B \pm \sigma_B) = (A+B) \pm ??$. 

**It is not $(A+B) \pm (\sigma_A + \sigma_B)$.** Can you explain why it isn't?

The standard way to propagate uncertainties in mathematical combinations of indepenent measurements (the value of $A$ in no way depends on the value of $B$), is as follows for the situation where $C = f(A,B)$ (and is generalizable to a larger number of variables):
$$\sigma_C^2 = \big(\frac{\partial f}{\partial A}\big)^2\sigma_A^2 + \big(\frac{\partial f}{\partial B}\big)^2\sigma_B^2$$

<div class="alert alert-info">Consider A $(12.5 \pm 3.5)$ and B $(13.1 \pm 2.9)$ and evaluate the sum and the difference of these, and given the propagated uncertainties using the above formula. Then, calculate the same using a numerical approach. Create two Gaussian samples of values representing A and B, sum (and difference) them and calculate the standard deviations of the resulting sets. Compare and discuss in the cell below.</div>

In [ ]:
# code to Monte Carlo the propagated uncertainties 
#
#

Depending on the context, sometimes we refer to the ratio between the measurement and the uncertainty as the **signal-to-noise ratio** (S/N). For example, if the measurement for A above ($12.5 \pm 3.4$) represented the measurement of the flux from a star, we would say that the star was detected with S/N = 3.7. 

<div class="alert alert-info">How does the S/N for an observed star vary with the time you observe the star? (assume constant flux with time). Answer below.</div>

In [ ]:
Answer:

Now lets consider why we do not do optical astronomy during the daytime - afterall, the stars themselves have the same flux day or night. The flux we observe for a star is the combination of the flux of the star and the flux of the sky. The flux of the sky is about $10^7$ times brighter in the daytime than at night. Lets assume that we have a star with flux 1000 units and at night the sky contributes 100 units on top of the star (so you measure 1100 units, subtract 100 units, and are left with a measureent of 1000 units).

<div class="alert alert-info"> What is the S/N of that observation?
What is the S/N obtained if one observes the same star during the day? Answer below. </div>

In [ ]:
Answer:

## Conclusion: 

 <div class="alert alert-info"> Save your notebook.  Append your LastNameFirstInitial to the filename and submit via D2L</div>